In [1]:
# pip install  yfinance==0.2.31 

In [2]:
# pip install wbdata

In [3]:
# Import / install relevant Python packages
import pandas as pd     
import pandas_datareader as pdr
from pandas_datareader import wb 
import yfinance as yf
import wbdata
import datetime as dt
import matplotlib.pyplot as plt

# Data Fetching:

In [6]:
# fetch data for all SP500 companies from 2010 to 2020 using yfinance 

def get_sp_data(start='2000-01-01', end=None):
    # Get the current SP components, and get a tickers list
    sp_wiki_tables = pd.read_html(
        'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    current_constituents = (sp_wiki_tables[0]['Symbol']
                            .str
                            .replace('.', '-')
                            .tolist())
                            
    past_constituents = (sp_wiki_tables[1]["Removed"]["Ticker"]
                            .dropna()
                            .drop_duplicates()
                            .str
                            .replace('.', '-')
                            .tolist())
    tickers = current_constituents + past_constituents
    # Download historical data to a multi-index DataFrame
    try:
        data = yf.download(tickers, start=start, end=end)
        filename = 'sp_components_data.pkl'
        data.to_pickle(filename)
        print('Data saved at {}'.format(filename))
    except ValueError:
        print('Failed download, try again.')
        data = None
    return data


if __name__ == '__main__':
    sp_data = get_sp_data()

[*********************100%%**********************]  802 of 802 completed


121 Failed downloads:
['LM', 'PBCT', 'ESV', 'QEP', 'FRC', 'DRE', 'HFC', 'DWDP', 'CXO', 'ALXN', 'VAR', 'RHT', 'ADS', 'WPX', 'TSS', 'RDC', 'LLL', 'NYX', 'MON', 'FBHS', 'MXIM', 'CELG', 'JCP', 'XL', 'FII', 'AKS', 'INFO', 'TIF', 'GRA', 'FRX', 'BBBY', 'LSI', 'XLNX', 'ENDP', 'FLIR', 'RRD', 'DF', 'KSU', 'WIN', 'NLSN', 'ETFC', 'STI', 'CERN', 'AVP', 'FDC', 'ARNC', 'XEC', 'TSG', 'WCG', 'CTXS', 'SIVB', 'FTR', 'RTN', 'YHOO', 'VIAB', 'APC', 'MNK', 'AGN', 'DTV', 'TWTR', 'NBL', 'DNR']: Exception('%ticker%: No timezone found, symbol may be delisted')
['KRFT', 'SIAL', 'FRE', 'LLTC', 'DPS', 'BRCM', 'BCR', 'PGN', 'TE', 'SMS', 'QTRN', 'TWC', 'HPH', 'MWW', 'LXK', 'RAI', 'WFM', 'GMCR', 'FDO', 'WYN', 'NOVL', 'SNDK', 'NVLS', 'CEPH', 'SWY', 'FNM', 'BS', 'BJS', 'LEH', 'JNS', 'ACAS', 'CMCSK', 'MIL', 'LVLT', 'EK', 'ARG', 'GENZ', 'CPGX', 'CVC', 'JOY', 'STJ', 'CFN', 'LDW', 'TYC', 'SPLS', 'XTO', 'CAM', 'DJ', 'APOL', 'BXLT', 'Q', 'HSP', 'GGP', 'LO', 'AV', 'MJN', 'PCS', 'JDSU', 'HCBK']: Exception('%ticker%: No price d


Data saved at sp_components_data.pkl


# Loading dataframe into a local PostgreSQL database table:

In [ ]:
import psycopg2
import pickle

# Load data from a pickle file
with open('sp_components_data.pkl', 'rb') as file:
    data = pickle.load(file)

# Connect to your PostgreSQL database
conn = psycopg2.connect(
    dbname="esgdata",
    user="macbook",
    password="",
    host="localhost"
)

# Get column names from DataFrame
data = data.iloc[:10,:10]
column_names = data.columns.tolist()
for i in range(len(column_names)):
    col_name = '_'.join(column_names[i])
    col_name = col_name.replace(" ", "_")
    col_name = col_name.replace("-", "_")
    column_names[i] = col_name
    
# Create a cursor object
cursor = conn.cursor()

# Table name
table_name = "Test_ESG_data"

# Generate the SQL CREATE TABLE statement
columns_sql = ", ".join([f"{col} FLOAT" for col in column_names])
create_table_sql = f"CREATE TABLE IF NOT EXISTS {table_name} (Date DATE, {columns_sql})"
cursor.execute(create_table_sql)

insert_columns = ", ".join(column_names)
placeholders = ", ".join(['%s'] * len(column_names))
sql_insert = f"INSERT INTO {table_name} ({insert_columns}) VALUES ({placeholders})"

# Iterate over DataFrame rows and insert into the database
i=0
for index, row in data.iterrows():
    cursor.execute(sql_insert, tuple(row))
    i += 1
# Commit the transaction
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [2]:
    # Get the current SP components, and get a tickers list
    sp_wiki_tables = pd.read_html(
        'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    current_constituents = (sp_wiki_tables[0]['Symbol']
                            .str
                            .replace('.', '-')
                            .tolist())
                            
    past_constituents = (sp_wiki_tables[1]["Removed"]["Ticker"]
                            .dropna()
                            .drop_duplicates()
                            .str
                            .replace('.', '-')
                            .tolist())
    tickers = current_constituents + past_constituents
    # Download historical data to a multi-index DataFrame
#

/tmp/ipykernel_55042/1032448480.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  current_constituents = (sp_wiki_tables[0]['Symbol']
/tmp/ipykernel_55042/1032448480.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  past_constituents = (sp_wiki_tables[1]["Removed"]["Ticker"]


In [11]:
tickers_yahoo_yf = yf.Tickers(tickers[10:30])


In [13]:
help(tickers_yahoo_yf)

Help on Tickers in module yfinance.tickers object:

class Tickers(builtins.object)
 |  Tickers(tickers, session=None)
 |  
 |  Methods defined here:
 |  
 |  __init__(self, tickers, session=None)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __repr__(self)
 |      Return repr(self).
 |  
 |  download(self, period='1mo', interval='1d', start=None, end=None, prepost=False, actions=True, auto_adjust=True, repair=False, proxy=None, threads=True, group_by='column', progress=True, timeout=10, **kwargs)
 |  
 |  history(self, period='1mo', interval='1d', start=None, end=None, prepost=False, actions=True, auto_adjust=True, repair=False, proxy=None, threads=True, group_by='column', progress=True, timeout=10, **kwargs)
 |  
 |  news(self)
 |  
 |  ----------------------------------------------------------------------
 |  Data descriptors defined here:
 |  
 |  __dict__
 |      dictionary for instance variables (if defined)
 |  
 |  __weakref__
 |      list of we

In [157]:
yftickers   = 'META AMZN'                # yf format
tickers_yahoo_yf = yf.Tickers(yftickers)


In [151]:
display(df_yahoo_yf)

Adj Close                   Close                    High  \
                  AMZN        META        AMZN        META        AMZN   
Date                                                                     
1997-05-15    0.097917         NaN    0.097917         NaN    0.125000   
1997-05-16    0.086458         NaN    0.086458         NaN    0.098958   
1997-05-19    0.085417         NaN    0.085417         NaN    0.088542   
1997-05-20    0.081771         NaN    0.081771         NaN    0.087500   
1997-05-21    0.071354         NaN    0.071354         NaN    0.082292   
...                ...         ...         ...         ...         ...   
2023-10-11  131.830002  327.820007  131.830002  327.820007  132.050003   
2023-10-12  132.330002  324.160004  132.330002  324.160004  134.479996   
2023-10-13  129.789993  314.690002  129.789993  314.690002  133.309998   
2023-10-16  132.550003  321.149994  132.550003  321.149994  133.070007   
2023-10-17  131.470001  324.000000  131.470001  324.000000  132.580002   

                               Low                    Open              \
                  META        AMZN        META        AMZN        META   
Date                                                                     
1997-05-15         NaN    0.096354         NaN    0.121875         NaN   
1997-05-16         NaN    0.085417         NaN    0.098438         NaN   
1997-05-19         NaN    0.081250         NaN    0.088021         NaN   
1997-05-20         NaN    0.081771         NaN    0.086458         NaN   
1997-05-21         NaN    0.068750         NaN    0.081771         NaN   
...                ...         ...         ...         ...         ...   
2023-10-11  328.839996  129.610001  322.950012  129.740005  323.010010   
2023-10-12  330.540009  131.229996  322.690002  132.169998  328.000000   
2023-10-13  325.049988  128.949997  312.369995  132.979996  323.529999   
2023-10-16  321.820007  130.429993  315.519989  130.690002  318.640015   
2023-10-17  324.399994  128.710007  317.299988  130.389999  318.179993   

                Volume              
                  AMZN        META  
Date                                
1997-05-15  1443120000         NaN  
1997-05-16   294000000         NaN  
1997-05-19   122136000         NaN  
1997-05-20   109344000         NaN  
1997-05-21   377064000         NaN  
...                ...         ...  
2023-10-11    40741800  22036300.0  
2023-10-12    55528600  20530500.0  
2023-10-13    45786600  21341000.0  
2023-10-16    42832900  16536100.0  
2023-10-17    49316300  16372400.0  

[6650 rows x 12 columns]

In [5]:
yfperiod = 'max' 
for ikey in tickers_yahoo_yf.tickers.keys():
    print(ikey)
    display(yf.Ticker(ikey).balnce_sheet)  

A


AttributeError: 'Ticker' object has no attribute 'balnce_sheet'

In [10]:
yfperiod = 'max' 
for ikey in tickers_yahoo_yf.tickers.keys():
    print(ikey)
    display(yf.Ticker(ikey).)

A


YFNotImplementedError: Have not implemented fetching 'sustainability' from Yahoo API

In [16]:
yfperiod = 'max' 
for ikey in tickers_yahoo_yf.tickers.keys():
    print(ikey)
    display(yf.Ticker(ikey).get_info())

A


{'address1': '5301 Stevens Creek Boulevard',
 'city': 'Santa Clara',
 'state': 'CA',
 'zip': '95051',
 'country': 'United States',
 'phone': '800 227 9770',
 'fax': '866 497 1134',
 'website': 'https://www.agilent.com',
 'industry': 'Diagnostics & Research',
 'industryKey': 'diagnostics-research',
 'industryDisp': 'Diagnostics & Research',
 'sector': 'Healthcare',
 'sectorKey': 'healthcare',
 'sectorDisp': 'Healthcare',
 'longBusinessSummary': "Agilent Technologies, Inc. provides application focused solutions to the life sciences, diagnostics, and applied chemical markets worldwide. The Life Sciences and Applied Markets segment offers liquid chromatography systems and components; liquid chromatography mass spectrometry systems; gas chromatography systems and components; gas chromatography mass spectrometry systems; inductively coupled plasma mass spectrometry instruments; atomic absorption instruments; microwave plasma-atomic emission spectrometry instruments; inductively coupled plasm

ABNB


{'address1': '888 Brannan Street',
 'city': 'San Francisco',
 'state': 'CA',
 'zip': '94103',
 'country': 'United States',
 'phone': '415 510 4027',
 'website': 'https://www.airbnb.com',
 'industry': 'Travel Services',
 'industryKey': 'travel-services',
 'industryDisp': 'Travel Services',
 'sector': 'Consumer Cyclical',
 'sectorKey': 'consumer-cyclical',
 'sectorDisp': 'Consumer Cyclical',
 'longBusinessSummary': "Airbnb, Inc., together with its subsidiaries, operates a platform that enables hosts to offer stays and experiences to guests worldwide. The company's marketplace model connects hosts and guests online or through mobile devices to book spaces and experiences. It primarily offers private rooms, primary homes, or vacation homes. The company was formerly known as AirBed & Breakfast, Inc. and changed its name to Airbnb, Inc. in November 2010. Airbnb, Inc. was founded in 2007 and is headquartered in San Francisco, California.",
 'fullTimeEmployees': 6811,
 'companyOfficers': [{'ma

APD


...

{'address1': '1901 Chouteau Avenue',
 'city': 'Saint Louis',
 'state': 'MO',
 'zip': '63103',
 'country': 'United States',
 'phone': '314 621 3222',
 'website': 'https://www.ameren.com',
 'industry': 'Utilities - Regulated Electric',
 'industryKey': 'utilities-regulated-electric',
 'industryDisp': 'Utilities - Regulated Electric',
 'sector': 'Utilities',
 'sectorKey': 'utilities',
 'sectorDisp': 'Utilities',
 'longBusinessSummary': 'Ameren Corporation, together with its subsidiaries, operates as a public utility holding company in the United States. The company operates through four segments: Ameren Missouri, Ameren Illinois Electric Distribution, Ameren Illinois Natural Gas, and Ameren Transmission. It engages in the rate-regulated electric generation, transmission, and distribution activities; and rate-regulated natural gas distribution and transmission businesses. In addition, the company generates electricity through coal, nuclear, and natural gas, as well as renewable sources, suc

AAL


{'address1': '1 Skyview Drive',
 'city': 'Fort Worth',
 'state': 'TX',
 'zip': '76155',
 'country': 'United States',
 'phone': '682 278 9000',
 'website': 'https://www.aa.com',
 'industry': 'Airlines',
 'industryKey': 'airlines',
 'industryDisp': 'Airlines',
 'sector': 'Industrials',
 'sectorKey': 'industrials',
 'sectorDisp': 'Industrials',
 'longBusinessSummary': 'American Airlines Group Inc., through its subsidiaries, operates as a network air carrier. The company provides scheduled air transportation services for passengers and cargo through its hubs in Charlotte, Chicago, Dallas/Fort Worth, Los Angeles, Miami, New York, Philadelphia, Phoenix, and Washington, D.C., as well as through partner gateways in London, Doha, Madrid, Seattle/Tacoma, Sydney, and Tokyo. As of December 31, 2022, it operated a mainline fleet of 925 aircraft. The company was formerly known as AMR Corporation and changed its name to American Airlines Group Inc. in December 2013. American Airlines Group Inc. was f

AEP


{'address1': '1 Riverside Plaza',
 'city': 'Columbus',
 'state': 'OH',
 'zip': '43215',
 'country': 'United States',
 'phone': '614 716 1000',
 'website': 'https://www.aep.com',
 'industry': 'Utilities - Regulated Electric',
 'industryKey': 'utilities-regulated-electric',
 'industryDisp': 'Utilities - Regulated Electric',
 'sector': 'Utilities',
 'sectorKey': 'utilities',
 'sectorDisp': 'Utilities',
 'longBusinessSummary': 'American Electric Power Company, Inc., an electric public utility holding company, engages in the generation, transmission, and distribution of electricity for sale to retail and wholesale customers in the United States. It operates through Vertically Integrated Utilities, Transmission and Distribution Utilities, AEP Transmission Holdco, and Generation & Marketing segments. The company generates electricity using coal and lignite, natural gas, renewable, nuclear, hydro, solar, wind, and other energy sources. It also supplies and markets electric power at wholesale t

In [23]:
%pip install yesg

  Preparing metadata (setup.py) ... done
  Created wheel for yesg: filename=yesg-2.1.1-py3-none-any.whl size=6105 sha256=baf84c006dcf42420424b8d772bccea8cfa5de0904d892acddc7d2d38686561e
  Stored in directory: /home/miroslav/.cache/pip/wheels/77/d2/de/3487724c25c53b6cfc7b33cd6778732b831febaf37d1b1052e
Successfully built yesg
Note: you may need to restart the kernel to use updated packages.


In [24]:
import yesg

In [27]:
yesg.get_historic_esg('MSFT')

,Total-Score,E-Score,S-Score,G-Score
Date,,,,
2014-09-01,66.00,78.00,59.00,63.00
2014-10-01,66.00,78.00,59.00,63.00
2014-11-01,66.00,78.00,59.00,63.00
2014-12-01,66.00,78.00,59.00,63.00
2015-01-01,66.00,78.00,59.00,63.00
...,...,...,...,...
2022-04-01,NaN,NaN,NaN,NaN
2022-05-01,13.75,0.46,8.06,5.23
2022-06-01,NaN,NaN,NaN,NaN
